In [1]:
from dance.transforms.spatial_feature import SpatialIDEFeature

/home/zyxing/dance/dance/utils/matrix.py:135: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in numba.prange(n):


In [2]:
spatialIDEFeature=SpatialIDEFeature()

In [3]:
import pandas as pd
import anndata as ad
import numpy as np
from dance.transforms import AnnDataTransform

In [4]:
c_pd=pd.read_csv("spatialIDE_data/MouseDB/Rep11_MOB_0.csv",index_col=0)
xy_pd=pd.read_csv("spatialIDE_data/MouseDB/MOB_sample_info.csv",index_col=0)

In [5]:
c=ad.AnnData(c_pd.loc[c_pd.index.intersection(xy_pd.index)])
c.var_names_make_unique()
c.obs_names_make_unique()

xy=pd.read_csv("spatialIDE_data/MouseDB/MOB_sample_info.csv",index_col=0).loc[:,['x','y']]
c.obsm["spatial"] = xy.set_index(c.obs_names)

In [6]:
from dance.data.base import Data
d=Data(c)

In [7]:
from dance.transforms.filter import FilterGenesScanpy


transforms = [
            FilterGenesScanpy(min_counts=3),
            SpatialIDEFeature()
        ]

In [8]:
from dance.transforms.misc import Compose
transformer=Compose(*transforms, log_level="INFO")

In [9]:
transformer(d)

[INFO][2023-11-07 21:55:01,114][dance.Compose][__call__] Applying composed transformations:
Compose(
  FilterGenesScanpy(min_counts=3, min_cells=None, max_counts=None, max_cells=None, split_name=None),
  SpatialIDEFeature(),
)
[INFO][2023-11-07 21:55:01,119][dance.FilterGenesScanpy][__call__] Subsetting genes (-14,860 removed) due to FilterGenesScanpy(min_counts=3, min_cells=None, max_counts=None, max_cells=None, split_name=None)


In [10]:
ans=np.array(d.data.obsm['SpatialIDEFeature'])
ref=pd.read_csv("spatialIDE_data/MouseDB/ref.csv",index_col=0).to_numpy()

In [11]:
from scipy import stats


pers=[]
for row1,row2 in zip(ans,ref):
    pers.append(stats.pearsonr(row1,row2)[0])

In [12]:
np.mean(pers)

0.9999995862721741